Author: Maxime Marin  
@: mff.marin@gmail.com

# Accessing IMOS data case studies: Walk-through and interactive session

The next few notebooks aim to provide case studies using the cloud cluster availability demonstrated earlier, in order to investigate, load, visualise analyse and extract data stored on external servers.  
Main advantages include speed, ease of use and repeatability. Although we are using a python enviornment, novice python users can easily follow the receipes and adapt them to their needs.

This session will be divided into different notebooks focusing on different tasks including:

- <span style="color:green">**Notebook 1 - Start**:</span> Imports libraries, fires up the cluster, provides quick description of datasets available and loads dataset of interest.
- <span style="color:green">**Notebook 2 - Interactive**:</span> Provides interactive tools to investigate a the chosen dataset and make some quick plots.
- <span style="color:green">**Notebook 3 - Analysis**:</span> Performs further analysis on the chosen dataset including climatology, linear trends and anomalies.
- <span style="color:green">**Notebook 4 - Data Extraction**:</span> Extracts data into a format of choice and saves it locally for the user to perform more in-depth analysis.



***
## 1) LIBRARIES

In each notebooks, it is necessary to load the packages that are used within it to run the code. Note that some libraries might be called within other "bits" of code hidden in other files.  

"Importing" a library or package supposes that it was loaded in the python environment prior to running the notebooks... Thankfully we have taken care of that.  

It is customary to import and call libraries in the cell that they are used, or in a cell just before that. If a package is used throughout, we can call it in the first cell:


In [5]:
import sys
import os
sys.path.append('/home/jovyan/intake-aodn')

import intake_aodn #this library is part of the intake-aodn folder that we cloned and was created by us, containing functions we have created for the purpose of this project
import intake
from  intake_aodn.easicache import set_easi
intake_aodn.cat = set_easi()

/home/jovyan/intake-aodn/intake_aodn/catalogs/aodn_refs.zip aodn_refs.zip s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/aodn_refs.zip
/home/jovyan/intake-aodn/intake_aodn/catalogs/main.yaml main.yaml s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/main.yaml
/home/jovyan/intake-aodn/intake_aodn/catalogs/aodn.yaml aodn.yaml s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/aodn.yaml
/home/jovyan/intake-aodn/intake_aodn/catalogs/nci.yaml nci.yaml s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/nci.yaml


***
## 2) CLUSTER FIRING 

Let's now create a cluster to allow for a significant increase in speed.  
Of course, we first call the necessary libraries.

In [6]:
from intake_aodn.utils import get_distributed_cluster
client = get_distributed_cluster()
client

An existing cluster was found. Connected to cluster easihub.40971801a9254aa5aef7bf9e3ebf2af8


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.csiro.easi-eo.solutions/services/dask-gateway/clusters/easihub.40971801a9254aa5aef7bf9e3ebf2af8/status,


Notice how we imported `get_distributed_cluster` from `intake_aodn.utlis`. That is because `import` only maps the path, so we needed to tell jupyter where to find `get_distributed_cluster` first.  
utils however is the name of a file containing some functions (including `get_distributed_cluster`), so we can call it using a `.`.

Our cluster is stored under the variable `client`. Click on the dashboard link to see details about the cluster we have created.  

***
## 3) DATA DESCRIPTION 

Finally, let's have a look at what the available datasets are, along with some information about their metadata.

For this workshop, we have provided users with a choice between:
  
- IMOS SST: Satellite SST product created by IMOS hosted on the AODN servers.
- BRAN 2020: The CSIRO BlueLink Reanalysis product hosted on NCI. Only a few variables have been chosen for the sake of the workshop
- IMOS Chlorophyll: Satellite Chlorophyll product created by IMOS hosted on the AODN servers..
- SSTAARS: Sea Surface Temperature Australian Atlas of Regional Seas. A SST climatology product.

In [7]:
import ipywidgets as widgets
from ipywidgets import interactive, interact_manual
from intake_aodn.utils import display_entry 
from intake_aodn.utils import get_list_datasets

global catal # creates a varibale that is visible by all functions
catal = intake_aodn.cat

da_list,ser_list = get_list_datasets(catal) # gets list of severs and datasets

def display_detail(Dataset):
    display_entry(catal[ser_list[da_list.index(Dataset)]][Dataset])
    return Dataset

dst = interactive(display_detail,Dataset = da_list);
display(dst)

interactive(children=(Dropdown(description='Dataset', options=('SST_L3S_1d_ngt', 'SSTAARS_Daily_Climatology', …

The dropdown list includes all the datasets that are currently available (mapped) for the user.  
Outputs also show some basic information on each dataset, including a link to the catalogue of the hosting server. Those links contain a more complete list of paramters such as spatial coverage and lists of variables included in the files.

***
## 4) Region Selection & Loading

Once we have chosen the dataset we want to investigate, let's go ahead and select our region/location of interest. Note that at the moment, users can only download IMOS satellite products.

To help the user visualise where the region/location is located, the map below automatically updates as the user changes the text boxes defining min and max latitude and longitude.

In [ ]:
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, WMSLayer
from traitlets import Unicode

class WMSLayerCQL(WMSLayer):
    cql_filter = Unicode('').tag(sync=True, o=True)

p = Marker(location=(lat, lon))
m = Map(center=p.location, zoom=15)
wms = WMSLayerCQL(
    url='https://squidle.org/geoserver/squidle/wms?',
    layers='squidle:deployment_tracks,squidle:deployment_points',
    format='image/png', transparent=True, cql_filter='')

def set_pos(*args, **kwargs):  lat, lon = p.location

wms.cql_filter='key%3D236'
p.on_move(set_pos)
m.add_layer(p)
m.add_layer(wms)
display(m) 

In [8]:

from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
 MarkerCluster, Heatmap,SearchControl, 
 FullScreenControl)

import geopandas, pandas as pd, numpy as np
from ipywidgets import Text, HTML, IntSlider, jslink

center = (-32, 130)

m = Map(center=center, zoom=3,hight=100)

width_slider = IntSlider(description='Width(km):', min=0, max=100, value=20)
height_slider = IntSlider(description='Height(km):', min=0, max=100, value=20)
widget_width = WidgetControl(widget=width_slider, position='topright')
widget_height = WidgetControl(widget=height_slider, position='topright')

#jslink((zoom_slider, 'value'), (m, 'zoom'))
#widgetControl = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_width)
m.add_control(widget_height)

marker = Marker(location=center, draggable=True)
m.add_layer(marker);
m.layout.width = '50%'
m

Map(center=[-32, 130], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [9]:
from ipyleaflet import Map, Marker

center = (-32, 130)

m = Map(center=center, zoom=4)

marker = Marker(location=center, draggable=True)
m.add_layer(marker);

m

Map(center=[-32, 130], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [5]:
import cartopy
import cartopy.crs as ccrs
from ipywidgets import interactive
import matplotlib.pyplot as plt


def map_WA(lon_min,lon_max,lat_min,lat_max):
    lon = lon_min if lon_min == lon_max else [lon_min,lon_max]
    lat = lat_min if lat_min == lat_max else [lat_min,lat_max]

    fig = plt.figure(figsize=(30,8))
    ax = plt.axes(projection = ccrs.PlateCarree());
    ax.set_extent([90,140,-45,-5],crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(draw_labels=True,linestyle = '--')
    if isinstance(lon,list)  and isinstance(lat,list):
        ax.plot([lon[0],lon[0],lon[1],lon[1],lon[0]],[lat[0],lat[1],lat[1],lat[0],lat[0]],transform = ccrs.PlateCarree(),color='red')
    elif isinstance(lon,float)  and isinstance(lat,float):
        ax.scatter(lon,lat,s=55,marker="o",edgecolor = 'black',color = 'red',zorder=3)
    else:
        print('Selection cannot be a line')
    return lon, lat

w = interactive(map_WA,lon_min=widgets.FloatText(112.5),lon_max=widgets.FloatText(114.5),lat_min=widgets.FloatText(-27),lat_max=widgets.FloatText(-25));
display(w)

interactive(children=(FloatText(value=112.5, description='lon_min'), FloatText(value=114.5, description='lon_m…

Note that if the user indicates the same min-max latitude and longitude, it defines a point and places it on the map.  
However, selecting points along a line is not possible, so the map will not show anything in that case.  

To continue, let's select a region of your choice. Remember to not make it too big, we will see later how to "scale it up" (No more than 2x2 degrees!)

For the purpose of this workshop, I won't give you the choice: we will all download some satellite SST data provided by IMOS. Hey, you get to choose your region at least!  
NB: In future versions, users will be allowed to select any available product

Once we are happy with our choice of coordinates, let's load our data!

In [30]:
%%time 
from intake_aodn.utils import dw_data

if 'ds' in locals():
    del ds
    
coord = w.result
coord
print('Selected coordinates:' + str(coord))

entry=intake_aodn.cat.aodn_s3.SST_L3S_1d_ngt(startdt='1998-01-01',
                                             enddt='2021-01-01',
                                             cropto={'latitude':slice(max(coord[1]),min(coord[1])),'longitude':slice(min(coord[0]),max(coord[0]))})
ds = entry.to_dask()

ds

Selected coordinates:([112.5, 114.5], [-27.0, -25.0])
CPU times: user 3.16 s, sys: 78.5 ms, total: 3.24 s
Wall time: 19.6 s


<xarray.Dataset>
Dimensions:                  (time: 8312, latitude: 100, longitude: 100)
Coordinates:
  * latitude                 (latitude) float32 -25.01 -25.03 ... -26.97 -26.99
  * longitude                (longitude) float32 112.5 112.5 ... 114.5 114.5
  * time                     (time) datetime64[ns] 1998-01-01T15:20:00 ... 20...
Data variables:
    sses_bias                (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>
    l2p_flags                (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>
    sst_dtime                (time, latitude, longitude) float64 dask.array<chunksize=(1666, 100, 100), meta=np.ndarray>
    sses_count               (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>
    sea_surface_temperature  (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>
    sses_standard_deviation  (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>
    satellite_zenith_angle   (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>
    quality_level            (time, latitude, longitude) float32 dask.array<chunksize=(3346, 100, 100), meta=np.ndarray>

### Save the data to netcdf

In [31]:
ds =ds.compute()

In [32]:
ds

<xarray.Dataset>
Dimensions:                  (time: 8312, latitude: 100, longitude: 100)
Coordinates:
  * latitude                 (latitude) float32 -25.01 -25.03 ... -26.97 -26.99
  * longitude                (longitude) float32 112.5 112.5 ... 114.5 114.5
  * time                     (time) datetime64[ns] 1998-01-01T15:20:00 ... 20...
Data variables:
    sses_bias                (time, latitude, longitude) float32 0.06 ... nan
    l2p_flags                (time, latitude, longitude) float32 9.216e+03 .....
    sst_dtime                (time, latitude, longitude) float64 1.441e+04 .....
    sses_count               (time, latitude, longitude) float32 3.019 ... nan
    sea_surface_temperature  (time, latitude, longitude) float32 296.7 ... nan
    sses_standard_deviation  (time, latitude, longitude) float32 0.6626 ... nan
    satellite_zenith_angle   (time, latitude, longitude) float32 52.77 ... nan
    quality_level            (time, latitude, longitude) float32 5.0 5.0 ... nan

In [40]:
kelvintoc = 271
depthcorrection = 0.17
ds['SST_at_depth']= ds.sea_surface_temperature.where(ds['quality_level'] >=4) - ds.sses_bias + depthcorrection - kelvintoc
ds['SST_at_depth'].attrs ={'comment':'SST data with corrections apllied to depth',
                       'long name':'sea surface temperature at depth',
                       'unit':'degC'}

In [42]:
ds.to_netcdf('Example_Data.nc')

# Zarr work flow 

In [1]:
entry=intake_aodn.cat.aodn_s3.SST_L3S_1d_ngt(startdt='1998-01-01',
                                             enddt='2021-01-01',
                                             cropto={'latitude':slice(max(coord[1]),min(coord[1])),'longitude':slice(min(coord[0]),max(coord[0]))})
ds = entry.to_dask()
kelvintoc = 273.15
depthcorrection = 0.17
ds['SST_at_depth']= ds.sea_surface_temperature.where(ds['quality_level'] >=4) - ds.sses_bias + depthcorrection - kelvintoc
ds['SST_at_depth'].attrs ={'comment':'SST data with corrections apllied to depth',
                       'long name':'sea surface temperature at depth',
                       'unit':'degC'}

NameError: name 'intake_aodn' is not defined

In [ ]:
ds =ds.chunk({'time':100})

In [48]:
import fsspec
import boto3
import zarr
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
encoding = {vname: {'compressor': compressor} for vname in ds.variables}

scratch_bucket = "easihub-csiro-user-scratch"
s3 = boto3.client('s3')
userid = boto3.client('sts').get_caller_identity()['UserId']
dest =f"s3://{scratch_bucket}/{userid}/Eample_Data.zarr"

import s3fs
# List for 
fs = s3fs.S3FileSystem()
fs.rm(dest, recursive=True)

#s3.delete_object(Bucket=scratch_bucket,Key=f'{userid}/Eample_Data.zarr')
target = fsspec.get_mapper(dest)
ds.to_zarr(target,encoding=encoding, consolidated=True)

In [51]:
import xarray as xr
xr.open_zarr(target)

<xarray.Dataset>
Dimensions:                  (time: 8312, latitude: 100, longitude: 100)
Coordinates:
  * latitude                 (latitude) float32 -25.01 -25.03 ... -26.97 -26.99
  * longitude                (longitude) float32 112.5 112.5 ... 114.5 114.5
  * time                     (time) datetime64[ns] 1998-01-01T15:20:00 ... 20...
Data variables:
    SST_at_depth             (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    l2p_flags                (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    quality_level            (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    satellite_zenith_angle   (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    sea_surface_temperature  (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    sses_bias                (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    sses_count               (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    sses_standard_deviation  (time, latitude, longitude) float32 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>
    sst_dtime                (time, latitude, longitude) float64 dask.array<chunksize=(100, 100, 100), meta=np.ndarray>